## Creating the catalog with missing spectra (💩)

In [4]:
import os
import sys
import glob
import numpy as np
import scipy.optimize    as op
import matplotlib.pyplot as plt
import urllib.request
from astropy.io import fits as pyfits
from astropy.table import Table
from scipy.interpolate import make_interp_spline
import pandas as pd

local_dir="/home/colgoat/Jupyter/cat_fix14/"
file="DR14Q_prevlist.txt"
local_file = os.path.join(local_dir,file)
df = pd.read_csv(local_file,delim_whitespace=True, skiprows=0,\
dtype = {'folder':str,'spec':str,'plate':str,'mjd':str,'fiber':str,'fits':str})
                
t = Table.from_pandas(df)
t

miss = 0

for a in range(len(t)):
    if ((len(t['fiber'][a]) != 4) or ((len(t['plate'][a]) != 4))): #After or is for DR16
        miss = miss + 1
        print(str(a)+" -- "+t['plate'][a]+" "+t['mjd'][a]+" "+t['fiber'][a])
        
print(miss)
        
#len(t['fits'][0])



33 -- 3588 55184 02
40 -- 3588 55184 042
56 -- 3589 55186 012
88 -- 3590 55201 088
95 -- 3606 55182 033
110 -- 3606 55182 090
118 -- 3609 55201 064
125 -- 3609 55201 074
162 -- 3615 55445 010
164 -- 3615 55445 02
165 -- 3615 55445 072
167 -- 3615 55445 08
168 -- 3615 55445 080
175 -- 3615 56219 056
191 -- 3615 56544 092
209 -- 3647 55827 010
232 -- 3651 55247 018
251 -- 3655 55240 02
254 -- 3655 55240 026
256 -- 3655 55240 030
263 -- 3655 55240 040
272 -- 3655 55240 070
277 -- 3655 55240 074
283 -- 3655 55240 082
312 -- 3656 55241 040
315 -- 3656 55241 044
333 -- 3656 55241 072
353 -- 3657 55244 020
356 -- 3657 55244 026
363 -- 3657 55244 060
370 -- 3657 55244 068
389 -- 3658 55205 014
401 -- 3658 55205 039
412 -- 3659 55181 010
413 -- 3659 55181 018
426 -- 3659 55181 058
427 -- 3659 55181 064
441 -- 3659 55181 086
505 -- 3661 55614 056
508 -- 3661 55614 066
512 -- 3661 55614 08
525 -- 3662 55182 016
551 -- 3662 55182 046
555 -- 3662 55182 052
571 -- 3662 55182 074
577 -- 3662 55182 08

73811 -- 5174 56047 018
73823 -- 5174 56047 032
73830 -- 5174 56047 038
73841 -- 5174 56047 052
73844 -- 5174 56047 054
73884 -- 5174 56047 080
73900 -- 5174 56047 091
73908 -- 5174 56047 096
73921 -- 5175 55955 013
73927 -- 5175 55955 022
73933 -- 5175 55955 026
73942 -- 5175 55955 030
73958 -- 5175 55955 046
73962 -- 5175 55955 050
73971 -- 5175 55955 061
73978 -- 5175 55955 064
73982 -- 5175 55955 066
73988 -- 5175 55955 072
73993 -- 5175 55955 075
73998 -- 5175 55955 078
74019 -- 5175 55955 090
74025 -- 5175 55955 098
74027 -- 5176 56221 01000
74053 -- 5176 56221 026
74062 -- 5176 56221 031
74070 -- 5176 56221 040
74077 -- 5176 56221 044
74083 -- 5176 56221 048
74088 -- 5176 56221 054
74136 -- 5176 56221 096
74139 -- 5176 56221 098
74143 -- 5177 56245 012
74169 -- 5177 56245 034
74174 -- 5177 56245 036
74185 -- 5177 56245 048
74194 -- 5177 56245 057
74203 -- 5177 56245 063
74205 -- 5177 56245 064
74209 -- 5177 56245 065
74211 -- 5177 56245 066
74220 -- 5177 56245 070
74227 -- 5177 

135230 -- 6483 56341 030
135255 -- 6483 56341 044
135260 -- 6483 56341 052
135265 -- 6483 56341 058
135266 -- 6483 56341 06
135275 -- 6483 56341 072
135327 -- 6484 56358 018
135333 -- 6484 56358 024
135342 -- 6484 56358 028
135351 -- 6484 56358 032
135361 -- 6484 56358 040
135366 -- 6484 56358 044
135371 -- 6484 56358 048
135375 -- 6484 56358 054
135379 -- 6484 56358 06
135390 -- 6484 56358 068
135397 -- 6484 56358 076
135404 -- 6484 56358 084
135407 -- 6484 56358 086
135422 -- 6485 56342 012
135432 -- 6485 56342 026
135436 -- 6485 56342 028
135440 -- 6485 56342 032
135441 -- 6485 56342 034
135442 -- 6485 56342 036
135453 -- 6485 56342 043
135473 -- 6485 56342 06
135500 -- 6485 56342 094
135514 -- 6486 56328 020
135516 -- 6486 56328 024
135523 -- 6486 56328 027
135527 -- 6486 56328 03
135531 -- 6486 56328 031
135551 -- 6486 56328 042
135563 -- 6486 56328 052
135566 -- 6486 56328 053
135578 -- 6486 56328 068
135586 -- 6486 56328 071
135599 -- 6486 56328 080
135609 -- 6486 56328 09
13562

In [48]:
plate_list=[]
miss = 0

for a in range(len(t)):
    if (len(t['fiber'][a]) != 4 or ((len(t['plate'][a]) != 4))):
        miss = miss + 1
        if (len(t['fiber'][a]) == 5):
            t['fiber'][a] = t['fiber'][a][1:3]
                
        if (len(t['fiber'][a]) == 3):
            t['fiber'][a] = "0"+t['fiber'][a]

        if (len(t['fiber'][a]) == 2):
            t['fiber'][a] = "00"+t['fiber'][a]

        if (len(t['fiber'][a]) == 1):
            t['fiber'][a] = "000"+t['fiber'][a]
            
        if (len(t['plate'][a]) != 4):
            t['plate'][a] = "0"+t['plate'][a]
        
        plate_list.append(t['plate'][a]+'/spec-'+t['plate'][a]+"-"+t['mjd'][a]+"-"+t['fiber'][a]+"."+t['fits'][a])

        
#plate_list=np.unique(plate_list)
len(plate_list)
#print(miss)

17429

In [49]:
np.savetxt('/home/colgoat/Jupyter/cat_fix16/cnDR16Qdl.txt',plate_list, fmt='%s', delimiter=',')

## Correcting the main catalog removing misspelled names (🤡)

In [53]:
import os
import sys
import glob
import numpy as np
import scipy.optimize    as op
import matplotlib.pyplot as plt
import urllib.request
from astropy.io import fits as pyfits
from astropy.table import Table
from scipy.interpolate import make_interp_spline
import pandas as pd

local_dir="/home/colgoat/Jupyter/cat_fix16/"
file="DR16exct.txt"
local_file = os.path.join(local_dir,file)
df = pd.read_csv(local_file,delim_whitespace=True, skiprows=0,\
dtype = {'id':str})

miss = 0

initial_list=[]

tabla = Table.from_pandas(df)
tabla

for a in range(len(tabla)):
    if (len(tabla['id'][a]) != 30):
        miss = miss + 1
        #print(t['plate'][a]+" "+t['mjd'][a]+" "+t['fiber'][a])
    else:
        initial_list.append(tabla['id'][a])
               
np.savetxt('/home/colgoat/Jupyter/cat_fix16/rnDR16Qdl.txt',initial_list, fmt='%s', delimiter=',')

len(tabla) - miss

152408

## Checking the existing files on the web

In [100]:
import os
import urllib.request
import requests
import pandas as pd
from astropy.table import Table
import numpy as np
from astropy.io import fits as pyfits

local_dir = "/home/colgoat/Jupyter/cat_fixing/"
right_name = "rightnameDR14Qdownload.txt"
corre_name = "cornameDR14Qdownload.txt"

local_fileR = os.path.join(local_dir,right_name)
local_fileC = os.path.join(local_dir,corre_name)

dfR = pd.read_csv(local_fileR, sep=" ", header=None, skiprows=1,\
names = ['id'])
dfC = pd.read_csv(local_fileC, sep=" ", header=None, skiprows=1,\
names = ['id'])

GRn = [] #Group of right names as strings
GCn = [] #Group of corrected names as strings

for a in range(len(dfR)):
    row = dfR.iloc[a]
    ras = row.to_string(index=False) #row as string ras
    GRn.append(ras)

for b in range(len(dfC)):
    row = dfR.iloc[b]
    ras = row.to_string(index=False) #row as string ras
    GCn.append(ras)
    

print(len(GRn)," ",len(GCn))

152125   15920


In [104]:
import os
import urllib.request
import requests
import pandas as pd
from astropy.table import Table
import numpy as np
from astropy.io import fits as pyfits

#mainpath
mp = "https://data.sdss.org/sas/dr14/eboss/spectro/redux/v5_10_0/spectra/full/"
def check(path):
    r = requests.head(path)
    return r.status_code == requests.codes.ok


for a in range(70):
    if (check(mp+GRn[a]) != True):
        print(GRn[a])
    else:
        print(a)

print('done+')
#for b in range(len(GCn)):
    #if (check(mp+GCn[b]) == True):
        #missing.append('Found')
    #else:
        #missing.append('Not Found, check on the page')        


done+


In [108]:
mp = "https://data.sdss.org/sas/dr14/eboss/spectro/redux/v5_10_0/spectra/full/"
def check(path):
    r = requests.head(path)
    return r.status_code == requests.codes.ok


check(mp)

True